## Ironsworn

[Official site.](https://www.ironswornrpg.com/)

Roll 1d6 + modifier (action die) against two d10s (challenge dice).

* If the action die is > both challenge dice, it's a strong hit.
* If it is > than one challenge die, it's a weak hit.
* Otherwise, it's a miss.

Additionally, there is a momentum score.

* A positive momentum can be used to zero out any challenge die less than its value.
* A negative momentum zeroes out any action die equal to its value.

In [1]:
import piplite
await piplite.install("icepool")

import icepool
from icepool import d6, d10

## The classic mistake in implementing *Ironsworn*

Let's start without momentum and add it later.

The classic mistake with finding probabilities for *Ironsworn* is that this doesn't work:

In [2]:
def does_not_work(mod):
    return 2 @ (d6 + mod > d10)

print(does_not_work(1))

Die with denominator 3600

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |     1521 |  42.250000% |
|       1 |     1638 |  45.500000% |
|       2 |      441 |  12.250000% |




The trouble with this is that it effectively rolls two independent contests of d6 + modifier against d10, not using the same d6 against both.
This overestimates the chance of the central outcome (weak hit).

## Correct ways of implementing *Ironsworn*

A correct way to do this would be to use the `apply()` function to the three dice:

In [3]:
def ironsworn(a, mod, c1, c2):
    action_score = min(a + mod, 10)
    return (action_score > c1) + (action_score > c2)

def using_apply(mod):
    return icepool.apply(ironsworn, d6, mod, d10, d10)

print(using_apply(1))

Die with denominator 600

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |      271 |  45.166667% |
|       1 |      238 |  39.666667% |
|       2 |       91 |  15.166667% |




Or you can "roll" the action die first, and then use the `sub()` method to compare the result to two d10s.

In [4]:
def using_sub(mod):
    return (d6 + mod).clip(max_outcome=10).sub(lambda a: 2 @ (a > d10))

print(using_sub(1))

Die with denominator 600

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |      271 |  45.166667% |
|       1 |      238 |  39.666667% |
|       2 |       91 |  15.166667% |




## Momentum

Now let's add the momentum mechanic.

We can do this by modifying the action and challenge dice, which can also be done using the `sub()` method.

In [5]:
def using_sub(mod, momentum):
    action = d6
    challenge = d10
    if momentum > 1:
        challenge = challenge.sub(lambda c: 0 if c < momentum else c)
    if momentum < 0:
        # You can use a dict to map old outcomes to new outcomes.
        # Any outcomes not mentioned are preserved.
        action = action.sub({-momentum: 0})
    return (action + mod).sub(lambda a: 2 @ (a > challenge))

print(using_sub(1, 5))

Die with denominator 600

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |      185 |  30.833333% |
|       1 |      290 |  48.333333% |
|       2 |      125 |  20.833333% |




## Rerolls

How about the ability to reroll any or all of the dice (thrown simultaneously)? Brute-forcing the solution by trying all possible choices of rerolls on all possible initial rolls is not particularly efficient, but is good enough for a problem of this size.

We'll produce a table of the mean result for a given modifier.

In [ ]:
def ironsworn_optimal_reroll(a, mod, c1, c2):
    best_result = icepool.Die([0])
    for a_final in [a, d6]:
        for c1_final in [c1, d10]:
            for c2_final in [c2, d10]:
                result = icepool.apply(ironsworn, a_final, mod, c1_final, c2_final)
                if result.mean() > best_result.mean():
                    best_result = result
    return best_result

print('| Mod | Without reroll | With reroll |')
print('|----:|---------------:|------------:|')
for mod in range(-4, 10):
    without_reroll = icepool.apply(ironsworn, d6, mod, d10, d10)
    with_reroll = icepool.apply(ironsworn_optimal_reroll, d6, mod, d10, d10)
    print(f'| {mod} | {without_reroll.mean():0.3f} | {with_reroll.mean():0.3f} |')
    

| Mod | Without reroll | With reroll |
|----:|---------------:|------------:|
| -4 | 0.033 | 0.116 |
| -3 | 0.100 | 0.304 |
| -2 | 0.200 | 0.532 |
| -1 | 0.333 | 0.780 |
| 0 | 0.500 | 1.030 |
| 1 | 0.700 | 1.268 |
| 2 | 0.900 | 1.475 |
| 3 | 1.100 | 1.646 |
| 4 | 1.300 | 1.782 |
| 5 | 1.467 | 1.874 |
| 6 | 1.600 | 1.930 |
| 7 | 1.700 | 1.961 |
| 8 | 1.767 | 1.976 |
